In [174]:
import numpy as np
import pandas as pd
import itertools
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from itertools import product
from itertools import combinations_with_replacement
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
import math
%matplotlib inline

In [2]:
from sklearn.cluster import KMeans, SpectralClustering, AffinityPropagation
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
from utils import *
%load_ext autoreload
%autoreload 2

In [4]:
gj = pd.read_csv('./gj_df.csv')
# gj = gj[['ifp_id', 'ctt', 'cond', 'training', 'team', 'user_id', 'value', 'fcast_date']]
# gj['fcast_year'] = pd.to_datetime(gj['fcast_date']).dt.year
# gj['fcast_week'] = pd.to_datetime(gj['fcast_date']).dt.week
# gj['ifp_week'] = gj['fcast_year'].map(str) + gj['fcast_week'].map(str) + gj['ifp_id']
# gj = gj.drop('fcast_date', axis=1)
# gj = gj.drop_duplicates()
# gj.to_csv('./gj_df.csv', index=False)

In [52]:
# adult = pd.read_csv('../labels.txt', delimiter='\t', header=0, names=['user_id','website','rating'])
# trec = pd.read_csv('../trec-rf10-crowd/trec-rf10-data.txt', delimiter='\t')
# gj = pd.read_csv('./filled_active_df.csv')

# best_users = trec.groupby('workerID').count().sort_values('docID', ascending=False)[:150].index
# trec = trec[trec['workerID'].isin(best_users)]

# r = pd.Series([2,3,2,3], index=[1,2,0,-2])
# trec['label_bin'] = trec['label'].map(r)
# adult.rating.unique()

array(['G', 'P', 'R', 'X'], dtype=object)

In [22]:
ifp_var = gj.groupby('ifp_week')['value'].var().sort_values()
ifps_less_var = (ifp_var<.1).index
ifps_less_var

Index([u'2014311399-2', u'2014311244-0', u'2014311394-0', u'2015241543-0',
       u'2015231543-0', u'2015221520-0', u'2015211520-0', u'2015221543-0',
       u'2015201520-0', u'2015101520-0',
       ...
       u'2015141459-0', u'2015181417-0', u'2015151459-0', u'2015161459-0',
       u'2015171459-0', u'2015181459-0', u'2015191459-0', u'2015201459-0',
       u'2015211459-0', u'2015221459-0'],
      dtype='object', name=u'ifp_week', length=1548)

In [23]:
print gj

         ifp_id ctt  cond training  team  user_id  value  fcast_year  \
0        1244-0  1a     1        a   NaN       51   0.20        2015   
1        1244-0  1a     1        a   NaN       51   0.20        2015   
2        1244-0  1a     1        a   NaN       51   0.20        2015   
3        1244-0  1a     1        a   NaN       51   0.20        2015   
4        1244-0  1a     1        a   NaN       51   0.20        2015   
5        1244-0  1a     1        a   NaN       51   0.20        2015   
6        1244-0  1a     1        a   NaN       51   0.09        2015   
7        1244-0  1a     1        a   NaN       51   0.09        2015   
8        1394-0  1a     1        a   NaN       51   0.75        2014   
9        1394-0  1a     1        a   NaN       51   0.75        2014   
10       1394-0  1a     1        a   NaN       51   0.75        2014   
11       1394-0  1a     1        a   NaN       51   0.75        2014   
12       1394-0  1a     1        a   NaN       51   0.75        

In [56]:
# ratingcts = adult.groupby('user_id')['rating'].nunique()
# ratingcts = ratingcts[ratingcts==4]
# len(ratingcts)

389

In [7]:
# print(len(adult['user_id'].unique()))
# ratingcts = adult.groupby('user_id')['rating'].nunique()
# ratingcts = ratingcts[ratingcts==4]
# uids = ratingcts.index
# print(len(uids))
# adult = adult[adult['user_id'].isin(uids)]
# testframe = create_user_task_ids(adult, 'user_id', 'website', 'rating', prime=True)
# print(np.max(testframe['uid'].values))
testframe = create_user_task_ids(gj, 'user_id', 'ifp_week', 'value', False, True)
testframe = testframe[testframe['ifp_week'].isin(ifps_less_var)]
testframe.head()

,ifp_id,ctt,cond,training,team,user_id,value,fcast_year,fcast_week,ifp_week,bin,task_id,uid
0,1244-0,1a,1,a,NaN,51,0.2,2015,3,201531244-0,0.2,0,0
1,1244-0,1a,1,a,NaN,51,0.2,2015,4,201541244-0,0.2,1,0
2,1244-0,1a,1,a,NaN,51,0.2,2015,5,201551244-0,0.2,2,0
3,1244-0,1a,1,a,NaN,51,0.2,2015,6,201561244-0,0.2,3,0
4,1244-0,1a,1,a,NaN,51,0.2,2015,7,201571244-0,0.2,4,0


In [28]:
print(testframe.shape)
print testframe.columns
print(len(testframe['uid'].unique()))
print(len(testframe['task_id'].unique()))

(1499294, 13)
Index([u'ifp_id', u'ctt', u'cond', u'training', u'team', u'user_id', u'value',
       u'fcast_year', u'fcast_week', u'ifp_week', u'bin', u'task_id', u'uid'],
      dtype='object')
2052
1548


In [9]:
testframe['value'] = (testframe['value'] - .5)*2
# testframe['value'] = testframe['value']
# testframe['value'] = (testframe['value']*.5) + .5

In [24]:
testframe.head()

,ifp_id,ctt,cond,training,team,user_id,value,fcast_year,fcast_week,ifp_week,bin,task_id,uid
0,1244-0,1a,1,a,NaN,51,-0.6,2015,3,201531244-0,0.2,0,0
1,1244-0,1a,1,a,NaN,51,-0.6,2015,4,201541244-0,0.2,1,0
2,1244-0,1a,1,a,NaN,51,-0.6,2015,5,201551244-0,0.2,2,0
3,1244-0,1a,1,a,NaN,51,-0.6,2015,6,201561244-0,0.2,3,0
4,1244-0,1a,1,a,NaN,51,-0.6,2015,7,201571244-0,0.2,4,0


In [10]:
c = pd.cut(
    testframe['value'],
    [-np.inf, .2, .4, .6, .8, np.inf],
    labels=[2,3,5,7,11]
)
testframe['bin_levels'] = c

In [27]:
# testframe = testframe[testframe['uid']<1000]

In [13]:
def batcher(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def split(df):
    train_df, validate_df, test_df = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    return train_df, validate_df, test_df

class Model(nn.Module):
    def __init__(self, users, tasks, k=2):
        super(Model, self).__init__()
        self.user_lut = nn.Embedding(users, k)
        self.task_lut = nn.Embedding(tasks, k)

        self.user_bias = nn.Embedding(users, 1)
        self.task_bias = nn.Embedding(tasks, 1)
        self.global_bias = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, jokes):
        user_vectors = self.user_lut(users)
        task_vectors = self.task_lut(jokes)
        user_bias = self.user_bias(users)
        task_bias = self.task_bias(jokes)
        preds = torch.bmm(user_vectors.unsqueeze(1), task_vectors.unsqueeze(2)).squeeze() \
                         + user_bias.squeeze() + task_bias.squeeze() + self.global_bias.expand_as(user_bias.squeeze())
#         m = nn.Sigmoid()
        return preds
        
class Model2(nn.Module):
    def __init__(self, users, tasks, k=2):
        super(Model2, self).__init__()
        self.user_lut = nn.Embedding(users, k)
        self.task_lut = nn.Embedding(tasks, k)

        self.user_bias = nn.Embedding(users, 1)
        self.user_add_bias = nn.Embedding(users, 1)
        self.task_bias = nn.Embedding(tasks, 1)
        self.global_bias = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, jokes):
        user_vectors = self.user_lut(users)
        task_vectors = self.task_lut(jokes)
        user_bias = self.user_bias(users)
        task_bias = self.task_bias(jokes)
        user_add_bias = self.user_add_bias(users)

        preds = torch.bmm(user_vectors.unsqueeze(1), task_vectors.unsqueeze(2)).squeeze() \
                          + task_bias.squeeze() + self.global_bias.expand_as(user_bias.squeeze())
        m = nn.Tanh()
#         return m((user_bias.squeeze() * (preds - .5)) + .5)
#         return m(user_bias.squeeze() * preds )
        return m(user_bias.squeeze() * preds + user_add_bias.squeeze() )
            

def val(df, model):
    crit = nn.MSELoss(size_average=False)
    crit2 = nn.L1Loss(size_average=False)
    total_loss = 0.
    total_l1 = 0.
    total_num = 0
    for batch in batcher(df, 100):
        true_rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
        total_num = total_num + true_rating.size(0)
        users = Variable(torch.LongTensor(batch.uid.values))
        tasks = Variable(torch.LongTensor(batch.task_id.values))
#         print(users, tasks)
        scores = model.forward(users, tasks)
        total_loss += crit(scores, true_rating).data[0]
        total_l1 += crit2(scores,true_rating).data[0]
    return math.sqrt(total_loss/total_num), total_l1/total_num


def train(train_iter, val_iter, test_iter, model):
    opt = optim.SGD(model.parameters(), lr=.9)
    crit = nn.MSELoss()
    crit2 = nn.L1Loss()

    print("val:", val(validate_df, model))
    for epochs in range(1):
        avg_loss = 0
        avg_l1 = 0
        total = 0
        for i,batch in enumerate(batcher(train_df, 100)):
            opt.zero_grad()
            if total == 0:
                print batch.bin.values
            rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
#             print(rating)
            users = Variable(torch.LongTensor(batch.uid.values))
            tasks = Variable(torch.LongTensor(batch.task_id.values))
#             print(users, tasks)
            scores = model.forward(users, tasks) 
#             + torch.sum(model.user_lut.weight.data.pow(2)) + \
#             torch.sum(model.task_lut.weight.data.pow(2)) + torch.sum(model.user_bias.weight.data.pow(2)) + \
#             torch.sum(model.task_bias.weight.data.pow(2))
            loss = crit(scores, rating)
            #if i % 1000==0:
            #    print (loss.data[0])
            loss.backward()
            avg_loss += loss.data[0]
            avg_l1 += crit2(scores,rating).data[0]
            total += 1
            opt.step()
        print("train:", math.sqrt(avg_loss / float(total)), avg_l1/ float(total))
        print("val:", val(validate_df, model))
#         print(model.user_bias.weight.data)
    return model.user_lut.weight.data, model.user_bias.weight.data

In [32]:
train_df, validate_df, test_df = split(testframe)
validate_df = validate_df[validate_df['uid'].isin(train_df['uid'].values)][validate_df['task_id'].isin(train_df['task_id'].values)]

users = int(np.max(train_df.uid.unique()))
tasks = int(np.max(train_df.task_id.unique()))
print(users, tasks)
print(int(np.max(validate_df.uid.unique())))
print(int(np.max(validate_df.task_id.unique())))
model = Model2(users+1, tasks+1, k=4)
user_vec, user_bias = train(train_df, validate_df, test_df, model)

(2051, 1547)
2051
1547
('val:', (0.8875000975499351, 0.758736459664383))
         ifp_id   ctt  cond training  team  user_id  value  fcast_year  \
200164   1428-0    1p     1        p   NaN     8142  -0.34        2014   
278478   1467-2    1b     1        b   NaN     9735  -1.00        2015   
833967   1459-0    1z     1        z   NaN   125407   0.30        2015   
433576   1459-0    1a     1        a   NaN    22231   0.14        2014   
376032   1428-0    1a     1        a   NaN    21150  -0.34        2014   
539241   1419-2    1h     1        h   NaN    22599  -0.20        2014   
231171   1244-0   4b7     4        b   7.0     8811  -0.80        2014   
588708   1414-0  4b22     4        b  22.0    22871  -0.50        2014   
1443525  1529-0  4w74     4        w  74.0   145238  -1.00        2015   
537973   1480-0    1h     1        h   NaN    22580   0.14        2014   
1276099  1438-0    1z     1        z   NaN   130640   0.50        2014   
1295975  1417-0    1z     1        z   

In [121]:
user_vec, user_bias = model.user_lut.weight.data, model.user_bias.weight.data

In [26]:
first_import = pd.read_csv('./clean_frame.csv',delimiter='\t', low_memory=False)
first_import = first_import.drop(['Unnamed: 0'],axis=1)
print first_import
print first_import.columns

      user_id  gender_yr4  citizen_yr4  workhours_yr4  tradeexp_yr4  \
0        -1.0         1.0         -1.0            5.0           3.0   
1         2.0         1.0          1.0            4.0           2.0   
2         4.0         1.0          1.0            6.0           1.0   
3         5.0        -1.0          1.0            4.0           3.0   
4         7.0         1.0          1.0            4.0           3.0   
5         8.0         1.0          1.0            4.0           1.0   
6         9.0         1.0          1.0            2.0           3.0   
7        10.0         1.0          1.0            1.0           3.0   
8        11.0         1.0          1.0            1.0           2.0   
9        12.0         1.0         -1.0            6.0           2.0   
10       13.0         1.0          1.0            5.0           1.0   
11       14.0         1.0          1.0            5.0           3.0   
12       15.0         1.0         -1.0            4.0           2.0   
13    

In [39]:
joined_data = pd.merge(first_import,testframe,right_on='uid',left_on='user_id')
print joined_data

         user_id_x  gender_yr4  citizen_yr4  workhours_yr4  tradeexp_yr4  \
0              2.0         1.0          1.0            4.0           2.0   
1              2.0         1.0          1.0            4.0           2.0   
2              2.0         1.0          1.0            4.0           2.0   
3              2.0         1.0          1.0            4.0           2.0   
4              2.0         1.0          1.0            4.0           2.0   
5              2.0         1.0          1.0            4.0           2.0   
6              2.0         1.0          1.0            4.0           2.0   
7              2.0         1.0          1.0            4.0           2.0   
8              2.0         1.0          1.0            4.0           2.0   
9              2.0         1.0          1.0            4.0           2.0   
10             2.0         1.0          1.0            4.0           2.0   
11             2.0         1.0          1.0            4.0           2.0   
12          

In [40]:
print joined_data.shape
print testframe.shape
print first_import.shape

(1259152, 181)
(1499294, 13)
(1712, 168)


In [41]:
tot_feats = ['uid','task_id','value','bin']
tot_feats = tot_feats + list(first_import.columns.values)
print tot_feats
print len(tot_feats)

['uid', 'task_id', 'value', 'bin', 'user_id', 'gender_yr4', 'citizen_yr4', 'workhours_yr4', 'tradeexp_yr4', 'tradeenjoy_yr4', 'newshrs_yr4', 'raven_score_yr4', 'crt_score_yr4', 'crt_1_cor_yr4', 'crt_2_cor_yr4', 'crt_3_cor_yr4', 'crt_4_cor_yr4', 'crt_5_cor_yr4', 'crt_6_cor_yr4', 'crt_7_cor_yr4', 'crt_8_cor_yr4', 'crt_9_cor_yr4', 'crt_10_cor_yr4', 'crt_11_cor_yr4', 'crt_12_cor_yr4', 'crt_13_cor_yr4', 'crt_14_cor_yr4', 'crt_15_cor_yr4', 'crt_16_cor_yr4', 'crt_17_cor_yr4', 'crt_18_cor_yr4', 'numeracy_score_yr4', 'cpk_correct_1', 'cpk_correct_2', 'cpk_correct_3', 'cpk_correct_4', 'cpk_correct_5', 'cpk_correct_6', 'cpk_correct_7', 'cpk_correct_8', 'cpk_correct_9', 'cpk_correct_10', 'cpk_correct_11', 'cpk_correct_12', 'cpk_correct_13', 'cpk_correct_14', 'cpk_correct_15', 'cpk_correct_16', 'cpk_correct_17', 'cpk_correct_18', 'cpk_correct_19', 'cpk_correct_20', 'cpk_correct_21', 'cpk_correct_22', 'cpk_correct_23', 'cpk_correct_24', 'cpk_correct_25', 'cpk_correct_26', 'cpk_correct_27', 'cpk_corr

In [42]:
mask = joined_data.columns.isin(tot_feats)
cleanframe = joined_data[joined_data.columns[mask]]
print(cleanframe.shape)

(1259152, 171)


In [235]:
no_task = ['uid'] + list(first_import.columns.values)
mask_no_task = cleanframe.columns.isin(no_task)
small_frame = cleanframe[cleanframe.columns[mask_no_task]]
print small_frame.columns

Index([u'gender_yr4', u'citizen_yr4', u'workhours_yr4', u'tradeexp_yr4',
       u'tradeenjoy_yr4', u'newshrs_yr4', u'raven_score_yr4', u'crt_score_yr4',
       u'crt_1_cor_yr4', u'crt_2_cor_yr4',
       ...
       u'motivation_3', u'motivation_4', u'motivation_5', u'motivation_6',
       u'motivation_7', u'motivation_8', u'motivation_intrinsic_score',
       u'motivation_prosocial_score', u'motivation_extrinsic_score', u'uid'],
      dtype='object', length=168)


In [238]:
small_frame_unique = small_frame.drop_duplicates()
print small_frame_unique.shape

(1711, 168)


In [110]:
class Model_LightFM_Vanilla_Psych_With_Bias(nn.Module):
    def __init__(self, users, tasks, num_user_features, k=4):
        super(Model_LightFM_Vanilla_Psych_With_Bias, self).__init__()
        self.user_feature_lv = nn.Embedding(num_user_features, k)
        self.user_lv = nn.Embedding(users, k)
        self.task_lv = nn.Embedding(tasks, k)

        self.num_user_features_bias = nn.Embedding(num_user_features, 1)
        self.num_user_features_add_bias = nn.Embedding(num_user_features, 1)
        self.users_bias = nn.Embedding(users, 1)
        self.users_add_bias = nn.Embedding(users, 1)
        self.task_bias = nn.Embedding(tasks, 1)
        self.global_bias = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, users_features,jokes):
        user_vectors = self.user_lv(users)
        user_feature_vectors = torch.mm(users_features,self.user_feature_lv.weight)
        user_finals = torch.add(user_vectors,user_feature_vectors)
        task_vectors = self.task_lv(jokes)
        user_bias = self.users_bias(users)
        user_features_bias = torch.mm(users_features, self.num_user_features_bias.weight)
        task_bias = self.task_bias(jokes)
        user_add_bias = self.users_add_bias(users)
        user_features_add_bias = torch.mm(users_features,self.num_user_features_add_bias.weight)
        
        preds = torch.bmm(user_vectors.unsqueeze(1), task_vectors.unsqueeze(2)).squeeze() \
                          + task_bias.squeeze() + self.global_bias.expand_as(user_bias.squeeze())
        m = nn.Tanh()
#         return m((user_bias.squeeze() * (preds - .5)) + .5)
#         return m(user_bias.squeeze() * preds )
        return m((user_bias * user_features_bias).squeeze() * preds + (user_add_bias.squeeze() + user_features_add_bias.squeeze()))

In [ ]:
class Model_LightFM_Psych_First(nn.Module):
    def __init__(self, tasks, num_user_features, k=4):
        super(Model_LightFM_Psych_First, self).__init__()
        self.user_feature_lv = nn.Embedding(num_user_features, k)
        self.task_lv = nn.Embedding(tasks, k)

        self.num_user_features_add_bias = nn.Embedding(num_user_features, 1)
        self.task_bias = nn.Embedding(tasks, 1)
        self.global_bias = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, users_features,jokes):
        user_feature_vectors = torch.mm(users_features,self.user_feature_lv.weight)
        task_vectors = self.task_lv(jokes)
        user_features_add_bias = torch.mm(users_features, self.num_user_features_add_bias.weight)
        task_bias = self.task_bias(jokes)
        
        preds = torch.bmm(user_feature_vectors.unsqueeze(1), task_vectors.unsqueeze(2)).squeeze() \
                          + task_bias.squeeze() + self.global_bias.expand_as(task_bias.squeeze())
        m = nn.Tanh()
#         return m((user_bias.squeeze() * (preds - .5)) + .5)
#         return m(user_bias.squeeze() * preds )
        return m(preds + user_features_add_bias.squeeze())

class Model_LightFM_Users_Psych_Given(nn.Module):
    def __init__(self, users, tasks, feature_lv, feature_add_bias, task1_lv, task_bias1, global_bias1, k=4):
        super(Model_LightFM_Users_Psych_Given, self).__init__()
        self.user_lv = nn.Embedding(users, k)
        self.task2_lv = nn.Embedding(tasks, k)
        self.user_feature_lv = Variable(feature_lv.weight.data, requires_grad=False)
        self.task1_lv = nn.Embedding(tasks, k)
        self.task1_lv.weight = task1_lv.weight
        self.task1_lv.weight.requires_grad=False
        self.task_bias1 = nn.Embedding(tasks, 1)
        self.task_bias1.weight = task_bias1.weight
        self.task_bias1.weight.requires_grad=False
        self.global_bias1 = Variable(global_bias1.data, requires_grad=False)

        self.num_user_features_add_bias = Variable(feature_add_bias.weight.data, requires_grad=False)
        self.users_bias = nn.Embedding(users, 1)
        self.users_add_bias = nn.Embedding(users, 1)
        self.task_bias2 = nn.Embedding(tasks, 1)
        self.global_bias2 = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, users_features,jokes):
        user_vectors = self.user_lv(users)
        user_feature_vectors = torch.mm(users_features,self.user_feature_lv)
        user_finals = torch.add(user_vectors,user_feature_vectors)
        task_vectors1 = self.task1_lv(jokes)
        task_vectors2 = self.task2_lv(jokes)
        task_vectors = task_vectors1 + task_vectors2
        
        user_bias = self.users_bias(users)
        task_bias1 = self.task_bias1(jokes)
        task_bias2 = self.task_bias2(jokes)
        task_bias = task_bias1 + task_bias2
        user_add_bias = self.users_add_bias(users)
        global_bias = self.global_bias1 + self.global_bias2

        preds = torch.bmm(user_finals.unsqueeze(1), task_vectors.unsqueeze(2)).squeeze() \
                          + task_bias.squeeze() + global_bias.expand_as(user_bias.squeeze())
        m = nn.Tanh()
#         return m((user_bias.squeeze() * (preds - .5)) + .5)
#         return m(user_bias.squeeze() * preds )
        return m(user_bias.squeeze() * preds + user_add_bias.squeeze())

In [214]:
def batcher(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def split(df):
    train_df, validate_df, test_df = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    return train_df, validate_df, test_df

def val(df, model):
    crit = nn.MSELoss(size_average=False)
    crit2 = nn.L1Loss(size_average=False)
    total_loss = 0.
    total_l1 = 0.
    total_num = 0
    for batch in batcher(df, 100):
        true_rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
        total_num = total_num + true_rating.size(0)
        users = Variable(torch.LongTensor(batch.uid.values))
        tasks = Variable(torch.LongTensor(batch.task_id.values))
        users_features = Variable(torch.FloatTensor(batch.values[:,:167]))
#         print(users, tasks)
        scores = model.forward(users, users_features,tasks)
        total_loss += crit(scores, true_rating).data[0]
        total_l1 += crit2(scores,true_rating).data[0]
    return math.sqrt(total_loss/total_num), total_l1/total_num

def train(train_iter, val_iter, test_iter, model):
    train_params = itertools.ifilter(lambda p: p.requires_grad, model.parameters())
    train_param_list = [param for param in train_params]
    opt = optim.SGD(train_param_list, lr=.15)
    crit = nn.MSELoss()
    crit2 = nn.L1Loss()

    print("val:", val(validate_df, model))
    for epochs in range(150):
        avg_loss = 0
        avg_l1 = 0
        total = 0
        for i,batch in enumerate(batcher(train_df, 100)):
            opt.zero_grad()
            rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
#             print(rating)
            users = Variable(torch.LongTensor(batch.uid.values))
            users_features = Variable(torch.FloatTensor(batch.values[:,:167]))
            tasks = Variable(torch.LongTensor(batch.task_id.values))
#             print(users, tasks)
            scores = model.forward(users, users_features, tasks) 
#             + torch.sum(model.user_lut.weight.data.pow(2)) + \
#             torch.sum(model.task_lut.weight.data.pow(2)) + torch.sum(model.user_bias.weight.data.pow(2)) + \
#             torch.sum(model.task_bias.weight.data.pow(2))
            loss = crit(scores, rating)
            #if i % 1000==0:
            #    print (loss.data[0])
            loss.backward()
            avg_loss += loss.data[0]
            avg_l1 += crit2(scores,rating).data[0]
            total += 1
            opt.step()
        #if (epochs == 149 or epochs % 10 == 0):
        print("train:", math.sqrt(avg_loss / float(total)), avg_l1/ float(total))
        print("val:", val(validate_df, model))

In [219]:
train_df, validate_df, test_df = split(cleanframe)

In [209]:
users = int(np.max(train_df.uid.unique()))
tasks = int(np.max(train_df.task_id.unique()))
print(users, tasks)
print(int(np.max(validate_df.uid.unique())))
print(int(np.max(validate_df.task_id.unique())))
model_full = Model_LightFM_Vanilla_Psych_No_Bias(users+1, tasks+1, 167, k=4)
print "Psych No Bias"
train(train_df, validate_df, test_df, model_full)
print "Psych Features Only"
model_feat = Model_LightFM_Psych_First(tasks+1, 167, k=4)
train(train_df, validate_df, test_df, model_feat)
print "Psych Full"
model_full_divided = Model_LightFM_Users_Psych_Given(users+1, tasks+1, model_feat.user_feature_lv, \
                                                     model_feat.num_user_features_add_bias, \
                                        model_feat.task_lv, model_feat.task_bias, model_feat.global_bias, k=4)
train(train_df, validate_df, test_df, model_full_divided)

(2051, 1547)
2051
1547
Psych No Bias
('val:', (1.0708891941634497, 0.9741797762849089))
('train:', 1.0536613828868724, 0.9517919979373167)
('val:', (1.0405037372050918, 0.935224200660249))
('train:', 0.9046787080883812, 0.7590235442297889)
('val:', (0.9057850684136225, 0.759620731768661))
('train:', 0.8684427044387533, 0.7131518847022444)
('val:', (0.8730955341142729, 0.7189242722019326))
('train:', 0.8501069355975551, 0.6891454628017231)
('val:', (0.8672531253742205, 0.7119431811083785))
('train:', 0.8438784290802985, 0.6817723257544812)
('val:', (0.8505200417430913, 0.689745353282466))
('train:', 0.8399330535919071, 0.6759741127372656)
('val:', (0.8386203737921714, 0.6742538348536757))
('train:', 0.8342891620888174, 0.6686090140489456)
('val:', (0.8362331197395606, 0.6701849898016539))
('train:', 0.8323651627404135, 0.6660284563336918)
('val:', (0.8353831579425794, 0.6692333151830917))
('train:', 0.8289250578397462, 0.662760100350484)
('val:', (0.832575767227746, 0.6661581743654879))

In [239]:
def computeForward(model, users, users_features):
    user_vectors = model.user_lv(users)
    user_feature_vectors = torch.mm(users_features, model.user_feature_lv.weight)
    user_finals = torch.add(user_vectors,user_feature_vectors)
    return user_finals

def computePsychOnly(model, users, users_features):
    user_feature_vectors = torch.mm(users_features,model.user_feature_lv.weight)
    return user_feature_vectors
    
features = np.asarray(model_full.user_feature_lv.weight.data.numpy())
np.savetxt("psych_feature.csv", features, delimiter=",")
users = np.asarray(model_full.user_lv.weight.data.numpy())
np.savetxt("psych_users.csv", users, delimiter=",")
user_mul_bias = np.asarray(model_full.users_bias.weight.data.numpy())
np.savetxt("psych_users_mul.csv", user_mul_bias, delimiter=",")
user_add_bias = np.asarray(model_full.users_add_bias.weight.data.numpy())
np.savetxt("psych_users_add.csv", user_add_bias, delimiter=",")

users = Variable(torch.LongTensor(small_frame_unique.uid.values))
users_features = Variable(torch.FloatTensor(small_frame_unique.values[:,:167]))
user_vectors_np = computeForward(model_full,users,users_features).data.numpy()
np.savetxt("psych_user_lv.csv", user_vectors_np,delimiter=",")
user_vectors_psych_only_np = computePsychOnly(model_full,users,users_features).data.numpy()
np.savetxt("psych_only_user_lv.csv", user_vectors_psych_only_np,delimiter=",")

In [220]:
users = int(np.max(train_df.uid.unique()))
tasks = int(np.max(train_df.task_id.unique()))
print(users, tasks)
print(int(np.max(validate_df.uid.unique())))
print(int(np.max(validate_df.task_id.unique())))
model = Model_LightFM_Vanilla_Psych_No_Bias(users+1, tasks+1, 167, k=4)
train(train_df, validate_df, test_df, model)

(2051, 1547)
2051
1547
('val:', (1.067180925773829, 0.969334627918049))
('train:', 1.0867334668741573, 0.9933939020733105)
('val:', (1.0896133858949437, 0.9970832139945923))
('train:', 1.0905681403298861, 0.998148631031824)
('val:', (1.0906022864471456, 0.9981930936958698))
('train:', 1.0895606437057328, 0.9970900466866559)
('val:', (1.0903711654557373, 0.9979747790318545))


KeyboardInterrupt: 

In [62]:
print train_df.values[0]

[   1.    1.    4.    2.    4.    4.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.    1.   -1.    1.   -1.   -1.   -1.    1.    1.   -1.
    1.   -1.   -1.   -1.    1.    1.    1.    1.    1.   -1.    1.   -1.
    1.   -1.   -1.   -1.    1.    1.   -1.    1.    1.    1.   -1.    4.
    4.    4.    5.    1.    2.    1.    1.    3.    3.    4.    1.    1.
    3.    2.    2.    3.   -1.    1.   -1.   -1.   -1.    1.    1.    1.
   -1.    1.   -1.    1.   -1.    8.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   

In [64]:
print train_df.values[0,:167]

[ 1.  1.  4.  2.  4.  4. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.  1.  1. -1.
  1. -1. -1. -1.  1.  1.  1.  1.  1. -1.  1. -1.  1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1. -1.  4.  4.  4.  5.  1.  2.  1.  1.  3.  3.  4.  1.  1.
  3.  2.  2.  3. -1.  1. -1. -1. -1.  1.  1.  1. -1.  1. -1.  1. -1.  8.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]


In [63]:
print train_df.head()

         gender_yr4  citizen_yr4  workhours_yr4  tradeexp_yr4  tradeenjoy_yr4  \
698541          1.0          1.0            4.0           2.0             4.0   
296335          1.0          1.0            5.0           4.0             6.0   
1188589        -1.0          1.0            6.0           1.0             6.0   
866959         -1.0          1.0            4.0           3.0             2.0   
693909          1.0          1.0            4.0           5.0             5.0   

         newshrs_yr4  raven_score_yr4  crt_score_yr4  crt_1_cor_yr4  \
698541           4.0             -1.0           -1.0           -1.0   
296335           2.0              9.0           18.0            1.0   
1188589          3.0             -1.0           -1.0           -1.0   
866959           5.0             -1.0           -1.0           -1.0   
693909           3.0             -1.0           -1.0           -1.0   

         crt_2_cor_yr4  ...   motivation_6  motivation_7  motivation_8  \
698541      